In [1]:
##Code in this file has been modified from Boris Murmann's github below, and the open-source textbook from Harald Pretl and colleagues:
# https://github.com/bmurmann/Book-on-gm-ID-design/blob/main/starter_files_open_source_tools/gf180mcuD/techsweep_plots_from_mat.ipynb
# Murmann and his colleague have written a book on gm/id-based design, "Systematic Design of Analog CMOS Circuits" (2017)

#Pretl's Open-Source Book: https://iic-jku.github.io/analog-circuit-design/analog_circuit_design.pdf

# Copyright 2024 Harald Pretl
# Licensed under the Apache License, Version 2.0 (the “License”); you may not use this
# file except in compliance with the License. You may obtain a copy of the License at
# http://www.apache.org/licenses/LICENSE-2.0



# These OTAs were originally designed to interface with the outside pins. 
# However, the decision has been made to implement two-stage OTAs for the external devices.
# Instead, the 5T-OTAs will be used as internal components when needed.

import numpy as np
import scipy.constants as sc
import matplotlib.pyplot as plt
from pygmid import Lookup as lk

nfet = lk('../nfet_03v3.mat') #m3, m4
pfet = lk('../pfet_03v3.mat') # m1,2,5,6
VDS1 = 1.65

In [2]:
# define the given parameters as taken from the specification table or inital guesses
gm_id_m12 = 12.5
gm_id_m34 = 8
l_12 = 0.7
l_34 = 3


# Current Source Limitations
i_bias_in = 10e-6               #Reference current is 10uA. Note that if some internal design uses something like 30uA, you can mirror that value too
w_m6 = 3.5                      #UNIT PMOS WIDTH
l_56 = 1                        #UNIT PMOS LENGTH

# Performance Targets and Parameters
i_total = 10e-6
vin_min = 1
vin_max = 2.8
vdd_min = 3.2
vdd_max = 3.4

output_voltage = 1.2
f_bw = 840e3 # -3dB bandwidth of the voltage buffer
c_load = 2e-12 #Expected given parasitics


In [3]:
# we get the required gm of M1/2 from the -3dB bandwidth requirement of the voltage buff# note that the -3dB bandwidth of the voltage buffer with gain Av=1 is equal to the unit# of the ota, hence we wet them equal here
# we add a factor of 3 to allow for PVT variation plus additional MOSFET parasitic loadi
gm_m12 = gm_id_m12*(i_total/2)
print('gm12 =', round(gm_m12/1e-3, 4), 'mS')

# we round to 0.5µA bias currents
id_m12 = i_total/2



gm12 = 0.0625 mS


In [4]:
# we calculate the dc gain

#Intrinsic gain of mirrors and diff pair
gm_gds_m12 = pfet.lookup('GM_GDS', GM_ID=gm_id_m12, L=l_12, VDS=VDS1, VSB=0)
gm_gds_m34 = nfet.lookup('GM_GDS', GM_ID=gm_id_m34, L=l_34, VDS=VDS1, VSB=0)

#get diff pair gds values
gds_m12 = gm_m12 / gm_gds_m12
gm_m34 = gm_id_m34 * i_total/2
gds_m34 = gm_m34 / gm_gds_m34
print("ro34 =", round((1/gds_m34) /1000,2),"kOhm"  )

#get current mirror gds, gm
gm_m34 = gm_id_m34 * i_total/2
print("gm_34=", round(gm_m34/1e-3, 4), 'mS')
a0 = gm_m12 / (gds_m12 + gds_m34)
print('a0 =', round(20*np.log10(a0), 1), 'dB')


ro34 = 17503.76 kOhm
gm_34= 0.04 mS
a0 = 51.0 dB


In [5]:
# we calculate the MOSFET capacitance which adds to Cload, to see the impact on the BW
gm_cgs_m12 = pfet.lookup('GM_CGS', GM_ID=gm_id_m12, L=l_12, VDS=VDS1, VSB=0)
gm_cdd_m12 = pfet.lookup('GM_CDD', GM_ID=gm_id_m12, L=l_12, VDS=VDS1, VSB=0)
gm_cdd_m34 = nfet.lookup('GM_CDD', GM_ID=gm_id_m34, L=l_34, VDS=VDS1, VSB=0)
c_load_parasitic = abs(gm_m12/gm_cgs_m12) + abs(gm_m12/gm_cdd_m12) + abs(gm_m34/gm_cdd_m34)
print('Design tries to create 3x f_bw to preserve error margin:')
print('additional load capacitance =', round(c_load_parasitic/1e-15, 1), 'fF')
f_bw = gm_m12 / (4*np.pi * (c_load + c_load_parasitic))
print('unity gain bandwidth incl. parasitics =', round(f_bw/1e6, 2), 'MHz')

Design tries to create 3x f_bw to preserve error margin:
additional load capacitance = 33.2 fF
unity gain bandwidth incl. parasitics = 2.45 MHz


In [6]:
vgs_m12 = pfet.look_upVGS(GM_ID=gm_id_m12, L=l_12, VDS=VDS1, VSB=0.0)
vgs_m34 = nfet.look_upVGS(GM_ID=gm_id_m34, L=l_34, VDS=VDS1, VSB=0.0)
print('vgs_12 =', round(float(vgs_m12), 3), 'V')
print('vgs_34 =', round(float(vgs_m34), 3), 'V')




vgs_12 = 0.894 V
vgs_34 = 0.87 V


In [7]:
# calculate settling time due to slewing with the calculated bias current
t_slew = (c_load + c_load_parasitic) * output_voltage / i_total
print('slewing time =', round(t_slew/1e-6, 3), 'µs')
t_settle = 5/(2*np.pi*f_bw)
print('settling time =', round(t_settle/1e-6, 3), 'µs')

slewing time = 0.244 µs
settling time = 0.325 µs


In [8]:
# calculate voltage gain error
gain_error = a0 / (1 + a0)
print('voltage gain error =', round((gain_error-1)*100, 1), '%')

voltage gain error = -0.3 %


In [9]:
# calculate total rms output noise
sth_m12 = pfet.lookup('STH_GM', VGS=vgs_m12, L=l_12, VDS=VDS1, VSB=0) * gm_m12
gamma_m12 = sth_m12/(4*1.38e-23*300*gm_m12)
sth_m34 = nfet.lookup('STH_GM', VGS=vgs_m34, L=l_34, VDS=VDS1, VSB=0) * gm_m34
gamma_m34 = sth_m34/(4*1.38e-23*300*gm_m34)
output_noise_rms = np.sqrt(1.38e-23*300 / (c_load + c_load_parasitic) * (2*gamma_m12 + 2*gamma_m34*gm_m34/gm_m12))
print('output noise =', round(output_noise_rms/1e-6, 1), 'µVrms')


output noise = 74.1 µVrms


In [10]:
# calculate all widths
id_w_m12 = pfet.lookup('ID_W', GM_ID=gm_id_m12, L=l_12, VDS=vgs_m12, VSB=0)
w_12 = id_m12 / id_w_m12
w_12_round = max(round(w_12*2)/2, 0.5)
print('M1/2 W =', round(w_12, 2), 'um, rounded W =', w_12_round, 'um')

#M34
id_m34 = id_m12
id_w_m34 = nfet.lookup('ID_W', GM_ID=gm_id_m34, L=l_34, VDS=vgs_m34, VSB=0)
w_34 = id_m34 / id_w_m34
w_34_round = max(round(w_34*2)/2, 0.5)
print('M3/4 W =', round(w_34, 2), 'um, rounded W =', w_34_round, 'um')

#M5,6
w_ratio_m5 = (i_total / i_bias_in) 
id_w_m5 = i_total/(w_ratio_m5 * w_m6)
id_w_m6 = i_bias_in/w_m6
vgs_m56 = pfet.look_upVGS(ID_W=id_w_m5, L=l_56, VDS=VDS1, VSB=0.0)


w_5 = (w_ratio_m5 * w_m6)
w_5_round = max(round(w_5*2)/2, 0.5)
print('M5 W =', round(w_5, 2), 'um, rounded W =', w_5_round, 'um')

w_6_round = max(round(w_m6*2)/2, 0.5)
print('M6 W =', round(w_6_round, 2), 'um')

print(">>  [info] M6 was assumed to be a unit transistor. You MUST verify that the conclusions are reasonable:")

print("M5 W requires", round(w_ratio_m5,4), "unit transistors. Can you achieve this ratio?!")
print('vgs_56 =', round(float(vgs_m56), 3), 'V')

gm_id_5 = pfet.lookup('GM_ID', ID_W=id_w_m5, L=l_56, VDS=vgs_m56, VSB=0)
gm_id_5 = pfet.lookup('GM_ID', ID_W=id_w_m6, L=l_56, VDS=vgs_m56, VSB=0)
print("M5 GM/ID =", round(1*gm_id_5, 3), 'S/A')
print("M6 GM/ID =", round(1*gm_id_5, 3), 'S/A')


M1/2 W = 11.28 um, rounded W = 11.5 um
M3/4 W = 4.41 um, rounded W = 4.5 um
M5 W = 3.5 um, rounded W = 3.5 um
M6 W = 3.5 um
>>  [info] M6 was assumed to be a unit transistor. You MUST verify that the conclusions are reasonable:
M5 W requires 1.0 unit transistors. Can you achieve this ratio?!
vgs_56 = 1.202 V
M5 GM/ID = 4.356 S/A
M6 GM/ID = 4.356 S/A
